In [1]:
import os
import torch
import skimage
import pywt
import scipy.io
import scipy.signal
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy import stats
from einops import reduce, rearrange, repeat
from npeet import entropy_estimators as ee
from torch.optim.lr_scheduler import StepLR
from scipy.fft import rfft, rfftfreq, ifft
from einops import rearrange
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from Electrodes import Electrodes
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [2]:
class DEAPDatasetEEGFeatures(InMemoryDataset):
  def __init__(self, root, raw_dir, processed_dir, feature='de',target='participant_id', transform=None, pre_transform=None,include_edge_attr = True, undirected_graphs = True, add_global_connections=True, participant_from=1, participant_to=32, n_videos=40):
      self._raw_dir = raw_dir
      self._processed_dir = processed_dir
      self.participant_from = participant_from
      self.participant_to = participant_to
      self.n_videos = n_videos
      self.feature = feature
      self.target = target
      # Whether or not to include edge_attr in the dataset
      self.include_edge_attr = include_edge_attr
      # If true there will be 1024 links as opposed to 528
      self.undirected_graphs = undirected_graphs
      # Instantiate class to handle electrode positions
      print('Using global connections' if add_global_connections else 'Not using global connections')
      self.electrodes = Electrodes(add_global_connections, expand_3d = False)
      super(DEAPDatasetEEGFeatures, self).__init__(root, transform, pre_transform)
      self.data, self.slices = torch.load(self.processed_paths[0])
      
  @property
  def raw_dir(self):
      return f'{self.root}/{self._raw_dir}'

  @property
  def processed_dir(self):
      return f'{self.root}/{self._processed_dir}'

  @property
  def raw_file_names(self):
      raw_names = [f for f in os.listdir(self.raw_dir)]
      raw_names.sort()
      return raw_names

  @property
  def processed_file_names(self):
      if not os.path.exists(self.processed_dir):
        os.makedirs(self.processed_dir)
      file_name = f'{self.participant_from}-{self.participant_to}' if self.participant_from is not self.participant_to else f'{self.participant_from}'
      return [f'deap_processed_graph.{file_name}_{self.feature}_{self.target}.dataset']

  def process(self):
        # Number of nodes per graph
        n_nodes = len(self.electrodes.positions_3d)
        

        if self.undirected_graphs:
            source_nodes, target_nodes = np.repeat(np.arange(0,n_nodes),n_nodes), np.tile(np.arange(0,n_nodes),n_nodes)
        else:
            source_nodes, target_nodes = np.tril_indices(n_nodes,n_nodes)
        
        edge_attr = self.electrodes.adjacency_matrix[source_nodes,target_nodes]
        
        # Remove zero weight links
        mask = np.ma.masked_not_equal(edge_attr, 0).mask
        edge_attr,source_nodes,target_nodes = edge_attr[mask], source_nodes[mask], target_nodes[mask]

        edge_attr, edge_index = torch.FloatTensor(edge_attr), torch.tensor([source_nodes,target_nodes], dtype=torch.long)
        
        # Expand edge_index and edge_attr to match windows
        e_edge_index = edge_index.clone()
        e_edge_attr = edge_attr.clone()
        
        number_of_graphs = 5
        for i in range(number_of_graphs-1):
            a = edge_index + e_edge_index.max() + 1
            e_edge_index = torch.cat([e_edge_index,a],dim=1)
            e_edge_attr = torch.cat([e_edge_attr,edge_attr],dim=0)

        print(f'Number of graphs per video: {number_of_graphs}')
        # List of graphs that will be written to file
        data_list = []
        pbar = tqdm(range(self.participant_from,self.participant_to+1))
        for participant_id in pbar:
            raw_name = [e for e in self.raw_file_names if str(participant_id).zfill(2) in e][0]
            pbar.set_description(raw_name)
            # Load raw file as np array
            participant_data = scipy.io.loadmat(f'{self.raw_dir}/{raw_name}')
            signal_data = torch.FloatTensor(remove_baseline_mean(participant_data['data'][:,:32,:]))
#             signal_data = torch.FloatTensor()
            processed = []
            for i, video in enumerate(signal_data[:self.n_videos,:,:]):
                if self.feature == 'wav':
                    node_features = process_video_wavelet(video)
                elif self.feature =='wav-entropy':
                    node_features = process_video_wavelet(video, feature='entropy')
                else:
                    node_features = process_video(video, feature=self.feature)
                
                if self.target == 'emotion_labels':
                    target = [participant_data['labels'][i]]
                if self.target == 'participant_id':
                    target = participant_id-1
                elif self.target == 'video_id':
                    pass
                else:
                    raise 'Invalid target'
                data = Data(x=torch.FloatTensor(node_features),edge_attr=e_edge_attr,edge_index=e_edge_index, y=torch.LongTensor([target])) if self.include_edge_attr else Data(x=torch.FloatTensor(node_features), edge_index=e_edge_index, y=torch.LongTensor([target]))
                data_list.append(data) 
               
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [3]:
def calculate_de(window):
    return ee.entropy(window.reshape(-1,1), k=2)
# Input: Video with shape (32,7680)
# Output: Graph node features with shape (5*32, 59) -> 5 graphs with 32 nodes each with 59 features each
def process_video(video, feature='psd'):
    # Transform to frequency domain
    fft_vals = np.fft.rfft(video, axis=-1)
     # Get frequencies for amplitudes in Hz
    samplingFrequency = 128
    fft_freq = np.fft.rfftfreq(video.shape[-1], 1.0/samplingFrequency)
    # Delta, Theta, Alpha, Beta, Gamma
    bands = [(0,4),(4,8),(8,12),(12,30),(30,45)]
    
    band_mask = np.array([np.logical_or(fft_freq < f, fft_freq > t) for f,t in bands])
    band_mask = repeat(band_mask,'a b -> a c b', c=32)
    band_data = np.array(fft_vals)
    band_data = repeat(band_data,'a b -> c a b', c=5)
     
    band_data[band_mask] = 0
    
    band_data = np.fft.irfft(band_data)

    windows = skimage.util.view_as_windows(band_data, (5,32,128), step=128).squeeze()
    # (5, 32, 60, 128)
    windows = rearrange(windows, 'a b c d -> b c a d')
    
    if feature == 'psd':
        features = scipy.signal.periodogram(windows)[1]
        features = np.mean(features, axis=-1)
    elif feature == 'de':
        features = np.apply_along_axis(calculate_de, -1, windows)

    
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)

    return features

In [4]:
def remove_baseline_mean(signal_data):
    # Take first three senconds of data
    signal_baseline = np.array(signal_data[:,:,:128*3]).reshape(40,32,128,-1)
    # Mean of three senconds of baseline will be deducted from all windows
    signal_noise = np.mean(signal_baseline,axis=-1)
    # Expand mask
    signal_noise = repeat(signal_noise,'a b c -> a b (d c)',d=60)
    return signal_data[:,:,128*3:] - signal_noise

In [5]:
def process_video_wavelet(video, feature='energy', time_domain=False):
    band_widths = [32,16,8,4]
    features = []
    for i in range(5):
        if i == 0:
            # Highest frequencies (64-128Hz) are not used
            cA, cD = pywt.dwt(video.numpy(), 'db4')
        else:
            cA, cD = pywt.dwt(cA, 'db4')
            
            cA_windows = skimage.util.view_as_windows(cA, (32,band_widths[i-1]*2), step=band_widths[i-1]).squeeze()
            cA_windows = np.transpose(cA_windows[:59,:,:],(1,0,2))
            if feature == 'energy':
                cA_windows = np.square(cA_windows)
                cA_windows = np.sum(cA_windows, axis=-1)
                features.append(cA_windows)
            elif feature == 'entropy':
                cA_windows = np.square(cA_windows) * np.log(np.square(cA_windows))
                cA_windows = -np.sum(cA_windows, axis=-1)
                features.append(cA_windows)

            else:
                raise 'Error, invalid wavelet feature'
                
    if time_domain:
        features = np.transpose(features,(2,1,0))
    features = rearrange(features, 'a b c -> (a b) c')
    features = torch.FloatTensor(features)
    
    # Normalization
    m = features.mean(0, keepdim=True)
    s = features.std(0, unbiased=False, keepdim=True)
    features -= m
    features /= s
    return features

In [6]:
# Constants used to define data paths
ROOT_DIR = './'
RAW_DIR = 'data/matlabPREPROCESSED'
PROCESSED_DIR = 'data/graphProcessedData'

dataset = DEAPDatasetEEGFeatures(root= ROOT_DIR, raw_dir= RAW_DIR, processed_dir= PROCESSED_DIR, feature='de',participant_to=32)
# dataset = dataset.shuffle()

s01.mat:   0%|          | 0/32 [00:00<?, ?it/s]

Using global connections
Processing...
Number of graphs per video: 5


s32.mat: 100%|██████████| 32/32 [55:45<00:00, 104.55s/it]

Done!


In [7]:
# 880 used for training, 220 validation and 180 testing
# test_participant = 1
# 
splt_idx = 1100

dataset = dataset.shuffle()

# 85% used for train/val
train_dataset = dataset[:splt_idx]
test_dataset = dataset[splt_idx:]

len(train_dataset),len(test_dataset)

(1100, 180)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(device)

cuda


In [9]:
from torch_geometric.nn import GCN2Conv, GCNConv, global_max_pool as gmp
class Model(torch.nn.Module):
    def __init__(self, in_channels,n_graphs, hidden_channels=128, n_classes = 32):
        super(Model, self).__init__()
        
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        
        self.gconv1 = GCNConv(in_channels,hidden_channels*2)
        self.gconv2 = GCNConv(hidden_channels*2,hidden_channels)
        
#         self.gconv3 = GCNConv(in_channels,hidden_channels)
        
        # self.rnn = torch.nn.GRU(hidden_channels, rnn_hidden_dim, 2,dropout=0.2, batch_first=True)
        self.cnn1 = torch.nn.Conv1d(n_graphs, 1, kernel_size=1, stride=1)
        
        self.lin1 = torch.nn.Linear(32*hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, n_classes)

        self.softmax = nn.Softmax(dim=-1)

        
    def forward(self, batch):
        bs = len(torch.unique(batch.batch))
        x, edge_index, edge_attr = batch.x, batch.edge_index, batch.edge_attr
#         print(x.shape)
        x = self.gconv1(x, edge_index, edge_attr)
        x = self.gconv2(x, edge_index, edge_attr)
        x = F.dropout(x, p=0.4, training=self.training)
        x = x.relu()
#         print(x.shape)
        x = rearrange(x, '(bs g e) f -> (bs e) g f', bs=bs, e=32)
#         print(x.shape)
        x = self.cnn1(x).squeeze()
        x = x.tanh()
        x = rearrange(x, '(bs e) f -> bs (e f)', bs=bs)
#         print(x.shape)
#         x = torch.sum(x, dim=1)
        
        
        x = F.dropout(x, p=0.4, training=self.training)
#         print(x.shape)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
#         print(x.shape)
#         x = x.view(-1)
       
        x = self.softmax(x)
        return x

        

In [10]:
# %%timeit

model = Model(train_dataset[0].x.shape[1],train_dataset[0].x.shape[0]//32).to(device)  
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameter count: {pytorch_total_params}')

# model = model.to(devic)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=.1, rho=0.9, eps=1e-06, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(),lr=1e-2, weight_decay=0)
# optimizer = torch.optim.Adam(model.parameters(),lr=1e-4, weight_decay=1e-2)
# optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-5)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, weight_decay=5e-4)
optimizer = torch.optim.Adagrad(model.parameters(), lr=5e-3, lr_decay=1e-4, weight_decay=0)

# Instantiate optimizer
# scheduler = StepLR(optimizer, step_size=20, gamma=0.7)

# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()



def train(loader, target = 0):
    model.train()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        out = model(batch)
        loss = criterion(out,batch.y)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        pred = torch.argmax(out,-1)
        right += torch.sum((pred == batch.y).detach().cpu())
        tot += batch.y.shape[0]
        
    return np.array(losses).mean(), right/tot

def test(loader,verbose=False, target = 0):
    model.eval()
    losses = []
    right = 0
    tot = 0
    for batch in loader:
        batch = batch.to(device)
        # y = batch.y[:,target]
        out = model(batch)
        pred = torch.argmax(out,-1)
        if verbose:
            print(pred,batch.y)
        loss = criterion(out,batch.y)
        losses.append(loss.item())
        
        right += torch.sum((pred == batch.y).detach().cpu())
        tot += batch.y.shape[0]
    return np.array(losses).mean(), right/tot

best_val_loss = np.inf
esp = 0
MAX_ESP = 50

BS = 64

target = 0 # Valence-Arousal-Dominance-Liking

splt_idx = 1000
train_data, val_data = torch.utils.data.random_split(train_dataset, [splt_idx, len(train_dataset)-splt_idx])

train_loader = DataLoader(train_data, batch_size=BS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BS)
writer = SummaryWriter()
for epoch in range(1, 10000):    

    
    
    # Training and validation
    train_loss, train_acc = train(train_loader, target = target)
    val_loss, val_acc = test(val_loader , target = target)
    print(f'Epoch {epoch};t loss: {train_loss:.5f} ;t acc: {train_acc:.2f} ;v loss: {val_loss:.5f} ;v acc: {val_acc:.2f}')

    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/test', val_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Accuracy/test', val_acc, epoch)
    # Early stopping and checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        esp = 0
        torch.save(model.state_dict(),'./best_params') 
    else:
        esp += 1
        if esp >= MAX_ESP:
            break
            
    if epoch % 20 == 0:
        test_loader = DataLoader(test_dataset, batch_size=2)
        loss, acc = test(test_loader, True)
        print(f'Test loss: {loss} ; Test acc: {acc}')
        
    


print('Finished training')

Model parameter count: 577062
Epoch 1;t loss: 3.46826 ;t acc: 0.03 ;v loss: 3.49569 ;v acc: 0.01
Epoch 2;t loss: 3.44597 ;t acc: 0.06 ;v loss: 3.48259 ;v acc: 0.04
Epoch 3;t loss: 3.43457 ;t acc: 0.09 ;v loss: 3.47848 ;v acc: 0.03
Epoch 4;t loss: 3.41907 ;t acc: 0.10 ;v loss: 3.48020 ;v acc: 0.03
Epoch 5;t loss: 3.40850 ;t acc: 0.12 ;v loss: 3.48188 ;v acc: 0.03
Epoch 6;t loss: 3.39459 ;t acc: 0.13 ;v loss: 3.48287 ;v acc: 0.03
Epoch 7;t loss: 3.38613 ;t acc: 0.14 ;v loss: 3.48362 ;v acc: 0.03
Epoch 8;t loss: 3.37132 ;t acc: 0.16 ;v loss: 3.48429 ;v acc: 0.03
Epoch 9;t loss: 3.36386 ;t acc: 0.17 ;v loss: 3.48377 ;v acc: 0.03
Epoch 10;t loss: 3.36459 ;t acc: 0.16 ;v loss: 3.48152 ;v acc: 0.03
Epoch 11;t loss: 3.35391 ;t acc: 0.18 ;v loss: 3.47940 ;v acc: 0.03
Epoch 12;t loss: 3.34562 ;t acc: 0.19 ;v loss: 3.46388 ;v acc: 0.04
Epoch 13;t loss: 3.34102 ;t acc: 0.19 ;v loss: 3.46558 ;v acc: 0.04
Epoch 14;t loss: 3.33537 ;t acc: 0.20 ;v loss: 3.46806 ;v acc: 0.04
Epoch 15;t loss: 3.32637 ;t

Epoch 31;t loss: 3.29676 ;t acc: 0.22 ;v loss: 3.44100 ;v acc: 0.06
Epoch 32;t loss: 3.28712 ;t acc: 0.24 ;v loss: 3.43405 ;v acc: 0.09
Epoch 33;t loss: 3.29192 ;t acc: 0.24 ;v loss: 3.43352 ;v acc: 0.08
Epoch 34;t loss: 3.29069 ;t acc: 0.23 ;v loss: 3.42910 ;v acc: 0.09
Epoch 35;t loss: 3.29397 ;t acc: 0.24 ;v loss: 3.44172 ;v acc: 0.06
Epoch 36;t loss: 3.28557 ;t acc: 0.24 ;v loss: 3.44159 ;v acc: 0.06
Epoch 37;t loss: 3.28936 ;t acc: 0.24 ;v loss: 3.43994 ;v acc: 0.07
Epoch 38;t loss: 3.28183 ;t acc: 0.24 ;v loss: 3.43905 ;v acc: 0.07
Epoch 39;t loss: 3.28889 ;t acc: 0.24 ;v loss: 3.44774 ;v acc: 0.06
Epoch 40;t loss: 3.28188 ;t acc: 0.24 ;v loss: 3.44269 ;v acc: 0.07
tensor([21, 21], device='cuda:0') tensor([ 6, 27], device='cuda:0')
tensor([21, 16], device='cuda:0') tensor([14, 28], device='cuda:0')
tensor([21, 21], device='cuda:0') tensor([ 4, 21], device='cuda:0')
tensor([21, 17], device='cuda:0') tensor([14,  7], device='cuda:0')
tensor([21, 16], device='cuda:0') tensor([14,  7

tensor([13, 21], device='cuda:0') tensor([13, 31], device='cuda:0')
tensor([21, 16], device='cuda:0') tensor([23, 26], device='cuda:0')
tensor([ 5, 16], device='cuda:0') tensor([27, 14], device='cuda:0')
tensor([13, 21], device='cuda:0') tensor([13, 10], device='cuda:0')
tensor([16, 18], device='cuda:0') tensor([ 3, 18], device='cuda:0')
tensor([18, 16], device='cuda:0') tensor([ 8, 28], device='cuda:0')
tensor([21, 11], device='cuda:0') tensor([12, 11], device='cuda:0')
tensor([21, 21], device='cuda:0') tensor([3, 9], device='cuda:0')
tensor([17, 21], device='cuda:0') tensor([27,  6], device='cuda:0')
tensor([16, 16], device='cuda:0') tensor([ 5, 28], device='cuda:0')
tensor([21, 23], device='cuda:0') tensor([29, 26], device='cuda:0')
tensor([13, 21], device='cuda:0') tensor([14, 30], device='cuda:0')
tensor([16, 21], device='cuda:0') tensor([28, 25], device='cuda:0')
tensor([16, 23], device='cuda:0') tensor([26,  0], device='cuda:0')
tensor([ 5, 21], device='cuda:0') tensor([27,  5],

Epoch 81;t loss: 3.24343 ;t acc: 0.28 ;v loss: 3.43375 ;v acc: 0.08
Epoch 82;t loss: 3.24355 ;t acc: 0.28 ;v loss: 3.42837 ;v acc: 0.08
Epoch 83;t loss: 3.24246 ;t acc: 0.28 ;v loss: 3.43054 ;v acc: 0.08
Epoch 84;t loss: 3.23967 ;t acc: 0.28 ;v loss: 3.43046 ;v acc: 0.08
Epoch 85;t loss: 3.25025 ;t acc: 0.28 ;v loss: 3.43130 ;v acc: 0.08
Epoch 86;t loss: 3.23963 ;t acc: 0.28 ;v loss: 3.43087 ;v acc: 0.08
Epoch 87;t loss: 3.24180 ;t acc: 0.28 ;v loss: 3.43466 ;v acc: 0.08
Epoch 88;t loss: 3.23769 ;t acc: 0.28 ;v loss: 3.42928 ;v acc: 0.08
Epoch 89;t loss: 3.23626 ;t acc: 0.29 ;v loss: 3.43048 ;v acc: 0.08
Epoch 90;t loss: 3.23600 ;t acc: 0.28 ;v loss: 3.42913 ;v acc: 0.08
Epoch 91;t loss: 3.24112 ;t acc: 0.28 ;v loss: 3.42715 ;v acc: 0.08
Epoch 92;t loss: 3.23561 ;t acc: 0.29 ;v loss: 3.42434 ;v acc: 0.08
Epoch 93;t loss: 3.23629 ;t acc: 0.29 ;v loss: 3.42206 ;v acc: 0.08
Epoch 94;t loss: 3.24160 ;t acc: 0.28 ;v loss: 3.42077 ;v acc: 0.08
Epoch 95;t loss: 3.24239 ;t acc: 0.28 ;v loss: 3

Epoch 111;t loss: 3.22899 ;t acc: 0.30 ;v loss: 3.41620 ;v acc: 0.09
Epoch 112;t loss: 3.22568 ;t acc: 0.30 ;v loss: 3.41542 ;v acc: 0.09
Epoch 113;t loss: 3.22364 ;t acc: 0.31 ;v loss: 3.41059 ;v acc: 0.10
Epoch 114;t loss: 3.22352 ;t acc: 0.30 ;v loss: 3.42018 ;v acc: 0.09
Epoch 115;t loss: 3.22482 ;t acc: 0.31 ;v loss: 3.41005 ;v acc: 0.10
Epoch 116;t loss: 3.22105 ;t acc: 0.30 ;v loss: 3.40402 ;v acc: 0.10
Epoch 117;t loss: 3.22067 ;t acc: 0.31 ;v loss: 3.40228 ;v acc: 0.10
Epoch 118;t loss: 3.22660 ;t acc: 0.30 ;v loss: 3.39816 ;v acc: 0.11
Epoch 119;t loss: 3.22391 ;t acc: 0.30 ;v loss: 3.39992 ;v acc: 0.10
Epoch 120;t loss: 3.21713 ;t acc: 0.31 ;v loss: 3.39937 ;v acc: 0.11
tensor([21, 21], device='cuda:0') tensor([ 6, 27], device='cuda:0')
tensor([21, 16], device='cuda:0') tensor([14, 28], device='cuda:0')
tensor([21, 21], device='cuda:0') tensor([ 4, 21], device='cuda:0')
tensor([21, 17], device='cuda:0') tensor([14,  7], device='cuda:0')
tensor([17, 16], device='cuda:0') tens

...

tensor([13, 13], device='cuda:0') tensor([13, 31], device='cuda:0')
tensor([21,  5], device='cuda:0') tensor([23, 26], device='cuda:0')
tensor([ 5, 16], device='cuda:0') tensor([27, 14], device='cuda:0')
tensor([13, 21], device='cuda:0') tensor([13, 10], device='cuda:0')
tensor([17, 18], device='cuda:0') tensor([ 3, 18], device='cuda:0')
tensor([ 4, 16], device='cuda:0') tensor([ 8, 28], device='cuda:0')
tensor([17, 11], device='cuda:0') tensor([12, 11], device='cuda:0')
tensor([21, 21], device='cuda:0') tensor([3, 9], device='cuda:0')
tensor([ 5, 21], device='cuda:0') tensor([27,  6], device='cuda:0')
tensor([5, 4], device='cuda:0') tensor([ 5, 28], device='cuda:0')
tensor([21, 24], device='cuda:0') tensor([29, 26], device='cuda:0')
tensor([17, 21], device='cuda:0') tensor([14, 30], device='cuda:0')
tensor([16, 21], device='cuda:0') tensor([28, 25], device='cuda:0')
tensor([ 5, 23], device='cuda:0') tensor([26,  0], device='cuda:0')
tensor([5, 5], device='cuda:0') tensor([27,  5], dev

Epoch 321;t loss: 3.16738 ;t acc: 0.35 ;v loss: 3.32342 ;v acc: 0.17
Epoch 322;t loss: 3.16498 ;t acc: 0.35 ;v loss: 3.32295 ;v acc: 0.17
Epoch 323;t loss: 3.16705 ;t acc: 0.35 ;v loss: 3.32028 ;v acc: 0.18
Epoch 324;t loss: 3.16129 ;t acc: 0.35 ;v loss: 3.32099 ;v acc: 0.18
Epoch 325;t loss: 3.16435 ;t acc: 0.35 ;v loss: 3.32244 ;v acc: 0.17
Epoch 326;t loss: 3.16719 ;t acc: 0.35 ;v loss: 3.31910 ;v acc: 0.18
Epoch 327;t loss: 3.16333 ;t acc: 0.35 ;v loss: 3.31790 ;v acc: 0.18
Epoch 328;t loss: 3.16039 ;t acc: 0.36 ;v loss: 3.31881 ;v acc: 0.18
Epoch 329;t loss: 3.15857 ;t acc: 0.36 ;v loss: 3.31732 ;v acc: 0.18
Epoch 330;t loss: 3.16432 ;t acc: 0.36 ;v loss: 3.31716 ;v acc: 0.17
Epoch 331;t loss: 3.16120 ;t acc: 0.36 ;v loss: 3.32001 ;v acc: 0.17
Epoch 332;t loss: 3.15618 ;t acc: 0.37 ;v loss: 3.31749 ;v acc: 0.18
Epoch 333;t loss: 3.15858 ;t acc: 0.37 ;v loss: 3.31388 ;v acc: 0.19
Epoch 334;t loss: 3.15361 ;t acc: 0.37 ;v loss: 3.31360 ;v acc: 0.19
Epoch 335;t loss: 3.15716 ;t acc: 

Epoch 351;t loss: 3.14311 ;t acc: 0.38 ;v loss: 3.30985 ;v acc: 0.19
Epoch 352;t loss: 3.14819 ;t acc: 0.37 ;v loss: 3.31020 ;v acc: 0.19
Epoch 353;t loss: 3.15274 ;t acc: 0.37 ;v loss: 3.30851 ;v acc: 0.19
Epoch 354;t loss: 3.14240 ;t acc: 0.38 ;v loss: 3.30954 ;v acc: 0.19
Epoch 355;t loss: 3.14646 ;t acc: 0.37 ;v loss: 3.30845 ;v acc: 0.19
Epoch 356;t loss: 3.14370 ;t acc: 0.38 ;v loss: 3.30821 ;v acc: 0.19
Epoch 357;t loss: 3.14589 ;t acc: 0.38 ;v loss: 3.30771 ;v acc: 0.19
Epoch 358;t loss: 3.14732 ;t acc: 0.37 ;v loss: 3.30984 ;v acc: 0.19
Epoch 359;t loss: 3.14669 ;t acc: 0.38 ;v loss: 3.30924 ;v acc: 0.19
Epoch 360;t loss: 3.14297 ;t acc: 0.38 ;v loss: 3.30871 ;v acc: 0.19
tensor([21,  5], device='cuda:0') tensor([ 6, 27], device='cuda:0')
tensor([21, 16], device='cuda:0') tensor([14, 28], device='cuda:0')
tensor([21, 21], device='cuda:0') tensor([ 4, 21], device='cuda:0')
tensor([21, 16], device='cuda:0') tensor([14,  7], device='cuda:0')
tensor([17, 16], device='cuda:0') tens

tensor([21,  3], device='cuda:0') tensor([23, 26], device='cuda:0')
tensor([ 5, 17], device='cuda:0') tensor([27, 14], device='cuda:0')
tensor([13, 21], device='cuda:0') tensor([13, 10], device='cuda:0')
tensor([ 3, 18], device='cuda:0') tensor([ 3, 18], device='cuda:0')
tensor([24, 16], device='cuda:0') tensor([ 8, 28], device='cuda:0')
tensor([17, 11], device='cuda:0') tensor([12, 11], device='cuda:0')
tensor([21, 21], device='cuda:0') tensor([3, 9], device='cuda:0')
tensor([ 5, 24], device='cuda:0') tensor([27,  6], device='cuda:0')
tensor([5, 4], device='cuda:0') tensor([ 5, 28], device='cuda:0')
tensor([21,  3], device='cuda:0') tensor([29, 26], device='cuda:0')
tensor([17, 21], device='cuda:0') tensor([14, 30], device='cuda:0')
tensor([16, 21], device='cuda:0') tensor([28, 25], device='cuda:0')
tensor([ 5, 24], device='cuda:0') tensor([26,  0], device='cuda:0')
tensor([5, 5], device='cuda:0') tensor([27,  5], device='cuda:0')
tensor([16, 17], device='cuda:0') tensor([ 7, 14], dev

Epoch 401;t loss: 3.13999 ;t acc: 0.38 ;v loss: 3.31116 ;v acc: 0.19
Epoch 402;t loss: 3.14227 ;t acc: 0.38 ;v loss: 3.30955 ;v acc: 0.19
Epoch 403;t loss: 3.14011 ;t acc: 0.38 ;v loss: 3.30895 ;v acc: 0.19
Epoch 404;t loss: 3.14066 ;t acc: 0.38 ;v loss: 3.30963 ;v acc: 0.19
Epoch 405;t loss: 3.14208 ;t acc: 0.38 ;v loss: 3.31121 ;v acc: 0.19
Epoch 406;t loss: 3.13967 ;t acc: 0.38 ;v loss: 3.30911 ;v acc: 0.19
Epoch 407;t loss: 3.14117 ;t acc: 0.38 ;v loss: 3.30958 ;v acc: 0.19
Finished training


In [11]:
model.load_state_dict(torch.load('./best_params'))
test_loader = DataLoader(test_dataset, batch_size=1)
loss, acc = test(train_loader, False,target=target)
print(f'Train loss: {loss} ; Train acc: {acc}')
loss, acc = test(val_loader, False,target=target)

print(f'Val loss: {loss} ; Val acc: {acc}')
loss, acc = test(test_loader, False,target=target)
print(f'Test loss: {loss} ; Test acc: {acc}')

Train loss: 3.221319079399109 ; Train acc: 0.2980000078678131
Val loss: 3.3077125549316406 ; Val acc: 0.1899999976158142
Test loss: 3.329424171977573 ; Test acc: 0.18333333730697632
